<a href="https://colab.research.google.com/github/next-drought/ai-evaluation-workshops/blob/main/OpikOptimizerWorkshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smarter Prompting, Faster: Introducing Opik's Agent Optimizers

Doug Blank, Phd

* Slides are available at: [bit.ly/opik-optimizer-dsblank-slides](https://bit.ly/opik-optimizer-dsblank-slides)
* This notebook is available at: [bit.ly/opik-optimizer-dsblank](https://bit.ly/opik-optimizer-dsblank)

You will need:
1. A Google account, for running a Colab Notebook  - [google.com](https://google.com)
2. A Comet account, for seeing Opik visualizations (free!) - [comet.com](https://comet.com)
3. An OpenAI account, for using an LLM
[platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)


## Setup

This pip-install takes about a minute.

In [ ]:
%%capture
%pip install opik-optimizer

In [ ]:
import opik_optimizer
opik_optimizer.__version__

'0.7.3'

In [ ]:
import opik

# Configure Opik
opik.configure()

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········


OPIK: The API key provided is not valid on https://www.comet.com/. Please try again.


In [ ]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

To save time (and money) durring this demonstration, we have capture the results of a previous run of all of these LLM interactions. Our goal is to make this not cost any money. However, we of course can guarantee that. **Use at your own risk**!

To capture the perviously cached results:

In [ ]:
from opik_optimizer.demo.cache import get_litellm_cache

get_litellm_cache("opik-workshop")

## The Dataset

In these set of experiments, we are going to use the **HotPotQA** dataset. This dataset was designed to be difficult for regular LLMs to handle. This dataset is called a "**multi-hop**" dataset because answering the questions involves multiple reasoning steps and multiple tool calls, where the LLM needs to infer relationships, combine information, or draw conclusions based on the combined context.

Example:

> "What are the capitals of the states that border California?"

You'd need to find which states border California, and then lookup each state's capital.

The dataset has about 113,000 such crowd-sourced questions that are constructed to require the introductory paragraphs of two Wikipedia articles to answer.

[1] The name "HotPot" comes from the restaurant where the authors came up with the idea of the dataset.

In [ ]:
from opik_optimizer.demo import get_or_create_dataset

opik_dataset = get_or_create_dataset("hotpot-300")

Let's take a look at some dataset items:

In [ ]:
rows = opik_dataset.get_items()
rows[0]

In [ ]:
rows[1]

## Opik Project

All LLM traces in Opik are saved in a "project". We'll put them all in the following project name:

In [ ]:
project_name = "optimize-workshop-2025"

## The Metric

Choosing a good metric for optimization is tricky. For these examples, we'll pick one that will allow us to show improvement, and also provide a gradient of scores. In general though, this metric isn't the best for optimization runs.

We'll use "Edit Distance" AKA "Levenshtein Distance":

In [ ]:
from opik.evaluation.metrics import LevenshteinRatio
metric = LevenshteinRatio(project_name=project_name)

The metric takes two things: the output of the LLM and the reference (correct answer).

In [ ]:
metric.score("Hello", "Hello")

In [ ]:
metric.score("Hello!", "Hello")

The edit distance between "Hello!" and "Hello" is 1. Here is how the .91 is computed:

In [ ]:
edit_distance = 1

1 - edit_distance / (len("Hello1") + len("Hello"))


For more information see: [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance)

## Configuation

To create the necesary configurations for using an Opik Optimizer, you'll need three things:

1. An initial prompt
2. A MetricConfig
3. A TaskConfig

We're going to start with a pretty bad prompt... so we can optimize it!

In [ ]:
initial_prompt = "Provide an answer to the question"

The the two configurations:

In [ ]:
from opik_optimizer import (
    MetricConfig,
    TaskConfig,
    from_llm_response_text,
    from_dataset_field,
)

metric_config = MetricConfig(
    metric=LevenshteinRatio(project_name=project_name),
    inputs={
        "output": from_llm_response_text(),
        "reference": from_dataset_field(name="answer"),
    },
)

task_config = TaskConfig(
    instruction_prompt=initial_prompt,
    input_dataset_fields=["question"],
    output_dataset_field="answer",
    use_chat_prompt=True,
)

As you can see the MetricConfig is composed of our chosen metric. In addition, we need to know what the inputs will be. The inputs here are actually the outputs from the LLM.

We need two inputs for the metric:
1. The output produced by the LLM (uses a special name)
2. The correct answer (provided by the database item "answer")

The TaskConfig defines how to process a prompt. We need the initial prompt, and the inputs and outputs of the dataset.

In this case, we will use the chat_prompt format as our result.

## FewShotBayesianOptimizer

The FewShotBayesianOptimizer name indicates two things:

1. It will produce Chat Prompts, or FewShot examples as described in the slides.
2. Secondly, it describes how it searches for the best set of these FewShot examples.

To use this optimizer, we import it and create an instance, passing in the project name and model parameters:

In [ ]:
from opik_optimizer import (
    FewShotBayesianOptimizer,
)

optimizer = FewShotBayesianOptimizer(
    project_name=project_name,
    model="openai/gpt-4o-mini",
    temperature=0.1,
    max_tokens=5000,
)

### Baseline

Before we optimize this prompt ("Provide an answer to the question") let's see what the bare prompt does by itself on the dataset:

In [ ]:
score = optimizer.evaluate_prompt(
    dataset=opik_dataset,
    metric_config=metric_config,
    task_config=task_config,
    prompt=initial_prompt,
    n_samples=100,
)
score

It scored about 16% correct. [I say "percent correct" but because we are using edit distance, that isn't quite accurate. But we can think of it this way.]

Ok, let's optimize that prompt!

In [ ]:
result1 = optimizer.optimize_prompt(
    opik_dataset,
    metric_config,
    task_config,
    n_trials=3,
    n_samples=50
)

In [ ]:
result1.display()

What did we find? The result is a series of messages:

In [ ]:
result1.details["chat_messages"]

We'll see how we can use those in a few minutes.

## MetaPromptOptimizer

The MetaPromptOptimizer uses a clever idea: have the LLM generate better prompts!

Here is the internal system meta-prompt to have the LLM generate better prompts.

```text
You are an expert prompt engineer. Your task is to improve prompts for any type of task.

Focus on making the prompt more effective by:

1. Being clear and specific about what is expected
2. Providing necessary context and constraints
3. Guiding the model to produce the desired output format
4. Removing ambiguity and unnecessary elements
5. Maintaining conciseness while being complete

Return a JSON array of prompts with the following structure:
{
    "prompts": [
        {
            "prompt": "the improved prompt text",
            "improvement_focus": "what aspect this prompt improves",
            "reasoning": "why this improvement should help"
        }
    ]
}
```

This can work quite well on simpler datasets. It doesn't do so well on HotPot as we will see.

The MetaPromptOptimizer will try a number of rounds to try to find the best prompt.

In [ ]:
from opik_optimizer import (
    MetaPromptOptimizer,
)

optimizer = MetaPromptOptimizer(
    project_name=project_name,
    model="openai/gpt-4o-mini",  # Using gpt-4o-mini for evaluation for speed
    max_rounds=1,  # Number of optimization rounds
    num_prompts_per_round=2,  # Number of prompts to generate per round
    improvement_threshold=0.01,  # Minimum improvement required to continue
    temperature=0.1,  # Lower temperature for more focused responses
    max_completion_tokens=5000,  # Maximum tokens for model completion
    num_threads=1,  # Number of threads for parallel evaluation
    subsample_size=20,  # Fixed subsample size
)


We won't do too many rounds, as this is an impossible problem without tools.

In [ ]:
result2 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric_config=metric_config,
    task_config=task_config,
    auto_continue=False,
    n_samples=20,  # Explicitly set
    use_subsample=True,  # Force using subsample for evaluation rounds
)

In [ ]:
result2.display()

## MiproOptimizer

MIPRO (Multi-Iteration Prompt Optimization) is an optimizer algorithm that refines both prompts and few-shot examples in a multi-stage LLM program. It works by generating, evaluating, and refining prompts to improve language model performance. MIPRO is a more advanced method than simply "prompt hacking," offering real optimization of LLM workflows.

This sophisticated method optimizes both instructions and examples together. Using Bayesian optimization (like the FewShotBayesianOptimizer), it finds the best combinations of both elements. Through multiple testing rounds, it creates an optimized prompt that pairs effective instructions with relevant examples.

For thi first optimization, we aren't going to give it any tools to work with. Let's see how it works:

In [ ]:
from opik_optimizer import MiproOptimizer

optimizer = MiproOptimizer(
    model="openai/gpt-4o-mini",  # LiteLLM or OpenAI name
    project_name=project_name,
    temperature=0.1,
    num_threads=16,
)

Remember that we are still starting with the initial prompt:

In [ ]:
initial_prompt

In [ ]:
result3 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric_config=metric_config,
    task_config=task_config,
    n_samples=50,
)

In [ ]:
result3.display()

In [ ]:
result3.demonstrations

### Agent with Tools

Now we'll try with tools. This will allow multi-prompt optimization.

First, we need a tool. We'll use this one from DSPy:

In [ ]:
# Tools:
import dspy

def search_wikipedia(query: str) -> list[str]:
    """
    This agent is used to search wikipedia. It can retrieve additional details
    about a topic.
    """
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(
        query, k=3
    )
    return [x["text"] for x in results]

Let's test it out on a subject:

In [ ]:
search_wikipedia("Developmental Robotics")

And it is easy to add the tools to the config. Let's go!

In [ ]:
task_config.tools = [search_wikipedia]

result4 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric_config=metric_config,
    task_config=task_config,
    n_samples=50,
)

In [ ]:
result4.display()

In [ ]:
result4.demonstrations

## Using Optimized Prompts

Recall:

1. result1 - FewShotBayesianOptimizer
2. result2 - MetaPromptOptimizer
3. result3 - MiproOptimizer (no tools)
4. result4 - MiproOptimizer (with search_wikipedia)

How can we use the optimized results?

For the first one, recall that the fewshot examples are here:

In [ ]:
result1.details["chat_messages"]

So, once we have those we can do the following:

In [ ]:
from litellm.integrations.opik.opik import OpikLogger
import litellm
opik_logger = OpikLogger()
litellm.callbacks = [opik_logger]

def query(question, chat_messages):
    messages = chat_messages[:-1] # Cut off the last one
    # replace it with question in proper format:
    messages.append({'role': 'user', 'content': '{"question": "%s"}"}' % question})

    response = litellm.completion(
        model="gpt-4o-mini",
        temperature=0.1,
        max_tokens=5000,
        messages=messages,
    )
    return response.choices[0].message.content

In [ ]:
query("When was David Chalmers born?", result1.details["chat_messages"])

In [ ]:
query("What weighs more: a baby elephant or an SUV?", result1.details["chat_messages"])

If it says "elephant" that is not correct!

Let's try that same question with a tool:

In [ ]:
result = result4.details["program"](question="What weighs more: a baby elephant or an SUV?")
result.answer

Well done optimizer!

We'll now head back to the slides to summarize the workshop.

# Resources

1. [Opik Optimizer Workshop Slides](https://bit.ly/opik-optimizer-dsblank-slides)